# Check Chris' failing simulation

Chris:

> alpha-4_MD002Msun_RD20_VF1000_4_STR1_q04_g1  
> $T(r)=158.9\,\mathrm{K}\cdot\left(\frac{r}{AU}\right)^{-2/5}$  
> `v_frag = lambda N: 100.*10.**smoothstep(d.x/AU-d.x[d.r_snow[N]]/AU,0.25)`  
> 1 Msun, 0.2-1000 AU bei 300 Zellen

In [ ]:
import os
import shutil
import sys

import numpy as np
import matplotlib.pyplot as plt

from dustpy.sim.constants import AU, yr, M_sun, R_sun
from dustpy.sim.utils import bindFunction
from dustpy.sim import Simulation


def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')


if is_interactive():
    from IPython import get_ipython
    get_ipython().magic('matplotlib inline')

Create simulation and set parameters

In [ ]:
s = Simulation()

s.snapshots = np.logspace(2, np.log10(3e6), 200) * yr

s.ini.grid.Nr = 300
s.ini.grid.rmin = 0.2 * AU
s.ini.grid.rmax = 1000 * AU

s.ini.star.M = M_sun
s.ini.star.R = 2.5 * R_sun
s.ini.star.Teff = 4000.

Mdisk = 0.002 * M_sun

s.ini.dust.vFrag = 1000.0

s.ini.gas.TExp = 2. / 5.
s.ini.gas.SigmaR0 = 20 * AU
s.ini.gas.SigmaExp = 1
s.ini.gas.Sigma0 = Mdisk / (2 * np.pi * s.ini.gas.SigmaR0**2)

s.pars.gasAdvection = True
s.pars.excludeAttr = ['dust/jac', 'dust/cFrag', 'dust/cStick', 'dust/kFrag', 'dust/kStick', 'dust/vRel']

I keep the function definitions here in the notebook to have everything self contained. However the cell is written to file and imported from there to be pickleable.

In [ ]:
%%file functions.py
from dustpy.sim.constants import AU
import numpy as np

def smoothstep(x, w):
    """
    This file produces a "smoothed heaviside" function

    y = 1/2*exp(x/w)       if x <= 0
    y = 1-1/2*exp(-x/w)    if x >  0

    Arguments
    ---------
    x : array-like
        input x-array

    w : float
        width of the transition

    Output
    ------
    y : array-like
        the function at every given x value

    """
    nd = np.ndim(x)
    x = np.array(x, ndmin=1)
    y = np.zeros(np.size(x))

    for i, r in enumerate(x):
        if r <= 0.:
            y[i] = 0.5 * np.exp(r / w)
        else:
            y[i] = 1. - 0.5 * np.exp(-r / w)
    if nd == 0:
        return y[0]
    else:
        return y

def smooth_vfrag(sim):
    i_sl = np.abs(sim.gas.T-170).argmin()
    r_sl = np.interp(170,sim.gas.T[i_sl-1:i_sl+2],sim.grid.r[i_sl-1:i_sl+2])
    return 100.*10.**smoothstep(sim.grid.r-r_sl,0.25*AU)

In [ ]:
from functions import smooth_vfrag # noqa
bindFunction(s, 'fragmentationVelocities', smooth_vfrag)

Delete output (?) and initialize

In [ ]:
if os.path.isdir(s.pars.outputDir):
    yn = ''
    if not is_interactive():
        yn = 'y'
    while yn.lower() not in ['y', 'n']:
        yn = input('output directory exists - delete? ')
    if yn == 'y':
        print('deleting')
        shutil.rmtree(s.pars.outputDir, ignore_errors=True)
    else:
        print('keeping')
s.initialize()

In [ ]:
s.evolve()

If running as a script: stop here!

In [ ]:
if not is_interactive():
    import sys # noqa
    sys.exit(0)

## Analysis

In [ ]:
mg = np.trapz(2 * np.pi * s.grid.r * s.gas.Sigma, x=s.grid.r) / M_sun * np.exp(1)
md = np.trapz(2 * np.pi * s.grid.r * s.dust.Sigma.sum(-1), x=s.grid.r) / M_sun * np.exp(1)

In [ ]:
f, ax = plt.subplots()
ax.contourf(s.grid.r / AU, s.dust.a[0, :], np.log10(s.dust.Sigma).T, 30, vmin=-9)
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
from dustpy.plotting import plot # noqa
plot('output')